In [1]:
import os

import numpy as np
import pandas as pd


In [2]:

# txt_test_sample.txt

with open('Original_Forums.txt', mode='r', encoding='iso-8859-1') as f:
    
    # find CarType, MsgTitle, MsgAuther, MsgDate and Msg in file
    car_type = []    # car tyepe: Makt + model
    makes = []
    msg_date = []
    msg_title = []
    msg_author = []
    reviews = [] 
        
    while True:
            
        end_of_message = False  # message end flag init
        
        line = f.readline()
        if not line:
            break
            
        if len(line) == 1:
            # message session end when read blank line
            end_of_message = True
            
        if end_of_message:
            # when the message sessin end, append to review list
            reviews.append(msg)
            
        if line.startswith('CarType:'):
            # get car type line
            type_ = line.split(':')[-1]
            car_type.append(type_)
            try:
                makes.append(type_.split()[1])
            except IndexError:
                # error happened on '71-Mazda6'
                makes.append(type_.split('-')[1].replace('6', ''))
            msg = ''    # init msg      
        elif line.startswith('MsgTitle:'):
            msg_title.append(line.split(':')[-1])
        elif line.startswith('MsgAuther:'):
            msg_author.append(line.split(':')[-1])
        elif line.startswith('MsgDate:'):
            time = line.split('Date:')[-1]
            msg_date.append(time)  
        elif line.startswith(' ') and not end_of_message:
            msg += line
            
    df = pd.DataFrame()
    df['make'] = makes
    df['type'] = car_type
    df['title'] = msg_title
    df['author'] = msg_author
    df['date'] = msg_date
    df['review'] = reviews


for i in range(len(df.columns)):
    # rm '\n' in cells
    df[df.columns[i]] = df[df.columns[i]].apply(lambda x: str(x).replace('\n', ''))
    
# df.to_csv('original_review.csv', index=False)    

In [48]:
df.head()
df['make'].unique()

array(['Acura', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet',
       'Chrysler', 'Dodge', 'Dodge/Plymouth', 'Ford', 'Honda', 'Hyundai',
       'Infiniti', 'Jaguar', 'Kia', 'Lexus', 'Lincoln', 'Maserati',
       'Mazda', 'Mercedes-Benz', 'Mercedes', 'Mitsubishi', 'Nissan',
       'Oldsmobile', 'Pontiac', 'Saab', 'Saturn', 'Suzuki', 'Toyota',
       'Volkswagen', 'Volvo'], dtype=object)

In [49]:
# 6 brands for compare, and each brands saved as individual .csv
makes = 'Honda Toyota Nissan Lexus BMW Audi Acura'.split()
makes

for i in makes:
    temp = df[df['make'] == i]
    temp = temp.dropna(axis=0)
    temp.reset_index(inplace=True, drop=True)
    temp.to_csv('data/{}.csv'.format(i), index=None)

        
merged = pd.read_csv('data/Honda.csv')
merged = merged.append(pd.read_csv('data/Acura.csv'))
merged = merged.append(pd.read_csv('data/Toyota.csv'))
merged = merged.append(pd.read_csv('data/Nissan.csv'))
merged = merged.append(pd.read_csv('data/Lexus.csv'))
merged = merged.append(pd.read_csv('data/BMW.csv'))
merged = merged.append(pd.read_csv('data/Audi.csv'))
merged.reset_index(inplace=True, drop=True)
merged.to_csv('data/merged7.csv', index=None)
merged.describe()

,make,type,title,author,date,review
count,401536,401536,400977,401536,401536,400105
unique,7,26,219108,40650,297188,329972
top,Honda,45-Honda Honda Accord,,,,Thanks!
freq,92130,62635,15493,15695,15695,30


In [23]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92130 entries, 0 to 92129
Data columns (total 6 columns):
make      92130 non-null object
type      92130 non-null object
title     91978 non-null object
author    92130 non-null object
date      92130 non-null object
review    91916 non-null object
dtypes: object(6)
memory usage: 4.2+ MB


In [45]:
# merged = pd.read_csv('data/merged6.csv')
 
norm = {}  # count the num of brands reviews
for i in makes:
    temp = pd.read_csv('data/{}.csv'.format(i))
    norm['{}'.format(i)] = temp.shape[0]
print(norm)

{'Honda': 92130, 'Toyota': 55362, 'Nissan': 31877, 'BMW': 88721, 'Audi': 31438, 'Acura': 46816}


In [47]:
for outer_make in makes:
    
    frequency = {}   # count comentioned 
    lift = {}
    temp = pd.read_csv('data/{}.csv'.format(outer_make))
    norm['{}'.format(outer_make)] = temp.shape[0]
    
    for inner_make in [make for make in makes if make != outer_make]:
        counter = 0   # counter for appearence
        for review in temp['review']:
            if inner_make in str(review):
                counter += 1
        combined_name = '{}_{}'.format(outer_make, inner_make)
        frequency[combined_name] = counter
        lift[combined_name+'_lift'] = \
                int(frequency[combined_name] / \
                    norm[combined_name.split('_')[0]] * norm[combined_name.split('_')[1]])
#     print(frequency)
    print(lift)

{'Honda_Toyota_lift': 2332, 'Honda_Nissan_lift': 440, 'Honda_BMW_lift': 1002, 'Honda_Audi_lift': 192, 'Honda_Acura_lift': 1009}
{'Toyota_Honda_lift': 6416, 'Toyota_Nissan_lift': 511, 'Toyota_BMW_lift': 1185, 'Toyota_Audi_lift': 182, 'Toyota_Acura_lift': 576}
{'Nissan_Honda_lift': 8497, 'Nissan_Toyota_lift': 3883, 'Nissan_BMW_lift': 1861, 'Nissan_Audi_lift': 323, 'Nissan_Acura_lift': 869}
{'BMW_Honda_lift': 1512, 'BMW_Toyota_lift': 687, 'BMW_Nissan_lift': 311, 'BMW_Audi_lift': 1304, 'BMW_Acura_lift': 1219}
{'Audi_Honda_lift': 2192, 'Audi_Toyota_lift': 1209, 'Audi_Nissan_lift': 409, 'Audi_BMW_lift': 12372, 'Audi_Acura_lift': 1992}
{'Acura_Honda_lift': 6812, 'Acura_Toyota_lift': 1135, 'Acura_Nissan_lift': 502, 'Acura_BMW_lift': 10335, 'Acura_Audi_lift': 1691}


In [ ]:
# sort
# 3-gram
# comments

1